In [66]:
import os
import pandas as pd

pred_path = '../n2c2-data/output'
gold_path = '../n2c2-data/test'
pred_path = '../ncbi-data/output'
gold_path = '../ncbi-data/test'

In [159]:
def getStats(df):
    "Gets stats for given dataframe"
    n = len(df)
    tp = sum(df.prediction == df.goldCui)
    fp = sum((df.prediction != df.goldCui) & (df.normalized==True))
    recall = round(tp/n,2) if n > 0 else 0
    precision = round(tp/(tp+fp),2) if (tp+fp) > 0 else 0
    return n, tp, fp, recall, precision

def sieveResults(results):
    "Returns a sieve-level analysis of results."
    levels = range(1,max(results.normalizingSieveLevel))
    sieves = pd.DataFrame([], columns=['sieve','n','tp','fp', 'recall', 'precision', 'agg_recall', 'agg_precision'])
    
    # Results for each sieve
    for i in levels:
        df = results[results.normalizingSieveLevel==i]
        n, tp, fp, recall, precision = getStats(df)
        sieve = df.normalizingSieveName.iloc[0] if n > 0 else "Unknown"
        sieves.loc[i] = [sieve, n, tp, fp, recall, precision, 0, 0]
        sieves.loc[i,'agg_recall'] = round(sum(sieves.tp)/sum(sieves.n),2)
        sieves.loc[i,'agg_precision'] = round(sum(sieves.tp)/(sum(sieves.tp)+sum(sieves.fp)),2)
    
    # Total results
    n, tp, fp, recall, precision = getStats(results)
    sieves.loc[i+1] = ['Total', n, tp, fp, '-', '-', recall, precision]
    return sieves

In [160]:
# Show results
results = pd.read_csv(f'{pred_path}/results.txt',sep='\t')
sieveResults(results)

,sieve,n,tp,fp,recall,precision,agg_recall,agg_precision
1,ExactMatchSieve,736,657,79,0.89,0.89,0.89,0.89
2,AbbreviationExpansionSieve,11,10,1,0.91,0.91,0.89,0.89
3,PrepositionalTransformSieve,2,2,0,1,1,0.89,0.89
4,Unknown,0,0,0,0,0,0.89,0.89
5,HyphenationSieve,3,1,2,0.33,0.33,0.89,0.89
6,AffixationSieve,3,3,0,1,1,0.89,0.89
7,DiseaseTermSynonymsSieve,16,9,3,0.56,0.75,0.88,0.89
8,StemmingSieve,3,2,1,0.67,0.67,0.88,0.89
9,Unknown,0,0,0,0,0,0.88,0.89
10,Total,964,685,88,-,-,0.71,0.89


In [36]:
files = os.listdir(pred_path)
i = 0
pred = pd.DataFrame([])
gold = pd.DataFrame([])
for file in files:
    if '.concept' in file:
        pred = pd.concat([pred, pd.read_table(f'{pred_path}/{file}',sep='\|\|',header=None, names=['file_id','ix','mention','cui'])])
        gold = pd.concat([gold, pd.read_table(f'{gold_path}/{file}',sep='\|\|',header=None, names=['file_id','ix','mention','cui'])])
        i += 1
print(i, len(pred),len(gold))
pred

C:\Users\etfrench\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys
C:\Users\etfrench\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


17 2063 2083


,file_id,ix,mention,cui
0,0034.concept,56|69,right le pain,NaN
1,0034.concept,527|531,pain,D010146
2,0034.concept,534|542,headache,C0018681
3,0034.concept,545|556,temperature,C0204688
4,0034.concept,617|626,oxycodone,NaN
...,...,...,...,...
74,974381789.concept,2273|2293,t-wave abnormalities,NaN
75,974381789.concept,2258|2268,st segment,NaN
76,974381789.concept,2250|2257,diffuse,C0205219
77,974381789.concept,2652|2670,septal hypertrophy,NaN


In [27]:
df = pd.read_csv('test4.txt',sep='\t',header=None)
df.to_csv('test4_pipe.txt',sep='|',header=None,index=None)

In [18]:
df[1] = df[1].replace('"','')
df.iloc[57]

0                                                   58
1    Fast thinker. \nVery efficient to work. Gets a...
Name: 57, dtype: object